<a href="https://colab.research.google.com/github/MiskirB/B5W6-Intelligent-Complaint-Analysis/blob/main/04_interactive_chat_interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 🧩 STEP 1: Install dependencies
!pip install -q streamlit pyngrok sentence-transformers faiss-cpu transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207

In [2]:
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted!")

# Optional: Verify your vector store path exists
# Example: !ls "/content/drive/MyDrive/B5W6-Intelligent-Complaint-Analysis/vector_store/

Mounted at /content/drive
Google Drive mounted!


In [3]:
from huggingface_hub import login
import os
from getpass import getpass

# Prompt for Hugging Face token (hidden input)
hf_token = getpass("🔐 Enter your Hugging Face token: ")

if hf_token:
    os.environ['HF_TOKEN'] = hf_token
    login(token=hf_token)
    print("✅ Logged in to Hugging Face successfully!")
else:
    print("❌ Hugging Face token not provided.")



🔐 Enter your Hugging Face token: ··········


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


✅ Logged in to Hugging Face successfully!


In [19]:
# %%writefile app.py # Keep this if you're writing to app.py in Colab

import gradio as gr
import pickle
import faiss
import numpy as np
import torch
import os
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from huggingface_hub import login

# --- Hugging Face Login ---
hf_token = os.environ.get("HF_TOKEN")
if hf_token:
    login(token=hf_token)

# File paths
INDEX_PATH = "/content/drive/MyDrive/B5W6-Intelligent-Complaint-Analysis/vector_store/faiss_index.index"
META_PATH = "/content/drive/MyDrive/B5W6-Intelligent-Complaint-Analysis/vector_store/metadata.pkl"

# Load resources
try:
    index = faiss.read_index(INDEX_PATH)
    with open(META_PATH, "rb") as f:
        metadata = pickle.load(f)
    embed_model = SentenceTransformer("all-MiniLM-L6-v2")
    generator = pipeline("text-generation", model="tiiuae/falcon-rw-1b", device=0 if torch.cuda.is_available() else -1)
except Exception as e:
    raise RuntimeError(f"Error loading resources: {e}")

def retrieve_similar_chunks(query, top_k=5):
    query_emb = embed_model.encode([query])[0].astype("float32")
    D, I = index.search(np.array([query_emb]), top_k)
    results = []
    for i, idx in enumerate(I[0]):
        entry = metadata[idx]
        results.append({
            "rank": i + 1,
            "product": entry["product"],
            "text": entry["text"]
        })
    return results

def format_prompt(chunks, question):
    context_blocks = "\n\n".join([f"[{i+1}] {c['text']}" for i, c in enumerate(chunks)])
    return f"""You are a financial analyst assistant for CrediTrust. Your task is to answer questions about customer complaints.

Use only the following retrieved complaint excerpts to formulate your answer. If the context does not contain enough information, respond with \"I don't have enough information to answer that.\"

Context:
{context_blocks}

Question: {question}

Answer:"""

def generate_response(prompt):
    result = generator(prompt, max_new_tokens=150, do_sample=True, temperature=0.7)[0]["generated_text"]
    return result.split("Answer:")[-1].strip() if "Answer:" in result else result.strip()

def answer_question(query):
    chunks = retrieve_similar_chunks(query)
    if not chunks:
        return "No relevant chunks found.", ""
    prompt = format_prompt(chunks, query)
    answer = generate_response(prompt)
    sources = "\n\n".join([f"🔹 {c['product']}:\n{c['text']}" for c in chunks])
    return answer, sources

with gr.Blocks() as demo:
    gr.Markdown("# 🧠 CrediTrust Complaint QA Chat")
    gr.Markdown("Ask any question related to customer complaints.")

    query_input = gr.Textbox(lines=2, label="💬 Ask your question about complaints:")
    answer_output = gr.Textbox(label="📌 Answer")
    source_output = gr.Textbox(label="📚 Retrieved Sources")

    feedback_msg = gr.Textbox(visible=False)  # optional confirmation message
    feedback_state = gr.State(value=None)  # stores the last answer to give feedback on

    with gr.Row():
        submit_btn = gr.Button("Ask")
        clear_btn = gr.Button("Clear")

    with gr.Row():
        like_btn = gr.Button("👍 Helpful")
        dislike_btn = gr.Button("👎 Not Helpful")

    def handle_query(query):
        answer, sources = answer_question(query)
        return answer, sources, answer  # store answer for feedback

    def give_feedback(feedback, answer):
        # Replace with logging, saving to file, or storing in db
        print(f"User feedback: {feedback} | Answer: {answer}")
        return f"✅ Feedback recorded: {feedback}"

    submit_btn.click(handle_query, inputs=[query_input], outputs=[answer_output, source_output, feedback_state])
    clear_btn.click(lambda: ("", "", None), outputs=[answer_output, source_output, feedback_state])

    like_btn.click(lambda ans: give_feedback("Helpful", ans), inputs=[feedback_state], outputs=[feedback_msg])
    dislike_btn.click(lambda ans: give_feedback("Not Helpful", ans), inputs=[feedback_state], outputs=[feedback_msg])

demo.launch(share=True)



Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
Device set to use cuda:0


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d73b91a15b4b8115a2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
